In [4]:
from flask import Flask, request, session, redirect, render_template

app = Flask("master_book")
app.secret_key = "secret_key"

# Database to store master and student login information
users = {
    "master": {"password": "password", "name": "Master User"},
    "student1": {"password": "password", "name": "Student 1"},
    "student2": {"password": "password", "name": "Student 2"}
}

@app.route("/")
def index():
    if "username" in session:
        return redirect("/home")
    return render_template("index.html")

@app.route("/login", methods=["POST"])
def login():
    username = request.form["username"]
    password = request.form["password"]
    if username in users and users[username]["password"] == password:
        session["username"] = username
        return redirect("/home")
    else:
        return "Invalid login. Please try again."

@app.route("/home")
def home():
    if "username" in session:
        return render_template("home.html", name=users[session["username"]]["name"])
    else:
        return redirect("/")

@app.route("/logout")
def logout():
    session.pop("username", None)
    return redirect("/")

if "master_book" == "_main_":
    app.run(debug=True)

## ANOTHER METHOD


In [6]:
from flask import Flask, render_template, request
import sqlite3

app = Flask("application_book")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/register', methods=['POST'])
def register():
    name = request.form['name']
    email = request.form['email']
    password = request.form['password']
    user_type = request.form['user_type']
    
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (name, email, password, user_type) VALUES (?, ?, ?, ?)", (name, email, password, user_type))
    conn.commit()
    cursor.close()
    conn.close()
    
    return render_template('register.html', name=name, email=email, user_type=user_type)

@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    password = request.form['password']
    
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE email=? AND password=?", (email, password))
    user = cursor.fetchone()
    cursor.close()
    conn.close()
    
    if user:
        return render_template('dashboard.html', name=user[0], user_type=user[4])
    else:
        return render_template('index.html', error="Invalid email or password")

@app.route('/masters')
def masters():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE user_type='Master'")
    masters = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return render_template('masters.html', masters=masters)

@app.route('/students')
def students():
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE user_type='Student'")
    students = cursor.fetchall()
    cursor.close()
    conn.close()
    
    return render_template('students.html', students=students)

if "appliation_book" == '_main_':
    app.run(debug=True)